In [ ]:
import scipy
import scipy.io as scio
from scipy import stats
from scipy import ndimage
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    #print("pts=",pts)
    leafsize = 2048
    # build kdtree
    #print 'build kdtree...'
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    #print 'query kdtree...' 
    distances, locations = tree.query(pts, k=4, eps=10.)

    #print 'generate density...'
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma =((distances[i][1]+distances[i][2]+distances[i][3])/3)*0.3
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point

        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    #print 'done.'
    return density

In [ ]:
for num in range(1,50):
    img = plt.imread('./ShanghaiTech/part_B/test_data/images/IMG_'+str(num)+'.jpg')
    mat = scio.loadmat('./ShanghaiTech/part_B/test_data/ground-truth/GT_IMG_'+str(num)+'.mat')
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0][0][0][0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])] = 1
    print("sum de k = ", np.sum(k))
    dens = gaussian_filter_density(k)
    np.save("GT_density_map/part_B/test_data/GT_density_IMG_" + str(num), dens)